This notebook presents a variety of tools for automatic optimization of Pytorch modules for inference.

You need at least the following requirements:

- fastai>=2.3.1
- opencv-python

In [ ]:
run_as_standalone_nb = True


from pathlib import Path
import sys


if run_as_standalone_nb:
    root_lib_path = Path('face2anime').resolve()
    if not root_lib_path.exists():
        !git clone https://github.com/davidleonfdez/face2anime.git
    if str(root_lib_path) not in sys.path:
        sys.path.insert(0, str(root_lib_path))
else:
    import local_lib_import

In [ ]:
import cv2
from face2anime.networks import CycleGenerator, default_decoder, default_encoder, Img2ImgGenerator
from face2anime.train_utils import FullyAveragedModel
from fastai.vision.all import *
import numpy as np
import pandas as pd
import timeit

# Helper methods

In [ ]:
def get_img_from_path(p):
    return cv2.imread(str(p))

def get_file_from_url(url):
    !wget $url
    path = url[len(url) - ''.join(reversed(url)).index('/'):]
    return path

def get_img_from_url(url):
    path = get_file_from_url(url)
    return get_img_from_path(path)

def bgr_to_rgb(img):
    return img[:,:,::-1]

def show_cv_img(img):
    plt.imshow(bgr_to_rgb(img))

# Load model

First, we are going to load the Pytorch module that we need to optimize.

Set `G_MODEL_PATH` to the path of the weights file of your model. It is assumed that is has been trained and saved with [this notebook](face2anime-bidirectional.ipynb).

In [ ]:
G_MODEL_PATH = Path('/kaggle/input/face2animeproduction/face2anime_bidir_17kds_tr2h2_425ep_ema.pth')
G_IMG_SZ = 64
G_NCH = 3
G_BS = 1
G_NORM_MEAN = 0.5
G_NORM_STD = 0.5


class GenDirection(Enum):
    FACE2ANIME = 0
    ANIME2FACE = 1


def get_dblock():
    normalize_tf = Normalize.from_stats(torch.tensor([G_NORM_MEAN]*3), torch.tensor([G_NORM_STD]*3))
    return DataBlock(blocks=(ImageBlock, ImageBlock),
                     get_items=get_image_files,
                     item_tfms=Resize(G_IMG_SZ, method=ResizeMethod.Crop),
                     batch_tfms=[normalize_tf],
                     splitter=IndexSplitter([]))


def get_dataloaders(path):
    return get_dblock().dataloaders(path, bs=1)


def remove_sn(module):
    for m in module.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
            torch.nn.utils.remove_spectral_norm(m)


def load_generator_learn(dls, img_size, n_channels, gen_dir=GenDirection.FACE2ANIME, mid_mlp_depth=2, 
                         norm=NormType.Batch, latent_sz=100, device='cpu'):
    def _decoder_builder(imsz, nch, latsz, hooks_by_sz=None): 
        return default_decoder(imsz, nch, latsz, norm_type=norm, hooks_by_sz=hooks_by_sz)
    generators = [Img2ImgGenerator(img_size, n_channels, mid_mlp_depth=mid_mlp_depth, skip_connect=True,
                                   encoder=default_encoder(img_size, n_channels, latent_sz, norm_type=norm),
                                   decoder_builder=_decoder_builder)
                  for _ in range(2)]
    generator = FullyAveragedModel(CycleGenerator(*generators))

    load_model(G_MODEL_PATH, generator, None, with_opt=False, device=device)
    g_module = generator.module.g_a2b if gen_dir == GenDirection.FACE2ANIME else generator.module.g_b2a
    learn = Learner(dls, g_module, loss_func=lambda *args: torch.tensor(0.))
    return learn

`ds_path` must be set to the parent path of an anime faces dataset.<br>
`celeba_path` must be set to the parent path of the CelebA dataset.

In [ ]:
ds_path = Path('/kaggle/input/animecharacterfaces/animeface-character-dataset/data')
celeba_path = Path('/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/')
ds_path.ls()

In [ ]:
dls = get_dataloaders(ds_path)
learn_f2a = load_generator_learn(dls, G_IMG_SZ, G_NCH, gen_dir=GenDirection.FACE2ANIME)
learn_a2f = load_generator_learn(dls, G_IMG_SZ, G_NCH, gen_dir=GenDirection.ANIME2FACE)

# Inference with fastai and Pytorch (used to compare results with optimized models)

In [ ]:
class FastaiInferenceEngine:
    def __init__(self, learn):
        self.learn = learn
        
    @torch.no_grad()    
    def __call__(self, cv_img):
        np_img = bgr_to_rgb(cv_img)
        dl = self.learn.dls.test_dl([np_img], bs=1, num_workers=0)
        inp, _, _, dec_preds = self.learn.get_preds(dl=dl, with_input=True, with_decoded=True)
        _, dec_preds = self.learn.dls.decode_batch((inp,)+tuplify(dec_preds))[0]
        return dec_preds.numpy()
        

def quick_encode(np_img):
    return ((TensorImage(np_img.copy()).permute(2, 0, 1).float()/255 - G_NORM_MEAN)/G_NORM_STD)[None]
        
        
class TorchQuickInferenceEngine:
    def __init__(self, model:nn.Module):
        self.model = model
        
    def _decode_torch_out(self, preds_t):
        return ((preds_t[0] * G_NORM_STD + G_NORM_MEAN) * 255).int()
        
    @torch.no_grad()
    def __call__(self, cv_img):
        downsample = cv_img.shape[0] > G_IMG_SZ
        # It's not clear what method is better when one dimension is bigger and the other is smaller,
        # so I'm only looking at height
        inter_method = cv2.INTER_AREA if downsample else cv2.INTER_LINEAR
        cv_img = cv2.resize(cv_img,
                            dsize=(G_IMG_SZ, G_IMG_SZ),
                            interpolation=inter_method)
        np_img = bgr_to_rgb(cv_img)
        inp = quick_encode(np_img)
        self.model.eval()
        preds_t = self.model(inp)
        dec_preds = self._decode_torch_out(preds_t) 
        return dec_preds.numpy()
    
    @classmethod
    def from_fastai_learn(cls, learn):
        return cls(learn.model)

In [ ]:
sample_img = get_img_from_path(celeba_path/'000002.jpg')
show_cv_img(sample_img)

In [ ]:
fastai_out = FastaiInferenceEngine(learn_f2a)(sample_img)
TensorImage(fastai_out).show()

In [ ]:
learn_f2a.model.eval()
torch_out = TorchQuickInferenceEngine.from_fastai_learn(learn_f2a)(sample_img)
TensorImage(torch_out).show()

Some differences between the PyTorch and the fastai output could appear due to different preprocessing.

# ONNX Runtime optimized model

ONNX is an open format built to represent machine learning models. There are converters from almost any machine learning framework to this format.

ONNX makes it easier to leverage hardware optimizations without needing them to be compatible with your prefered framework.

ONNX Runtime is an accelerator for machine learning models in ONNX format.

## Export to ONNX format

In [ ]:
def export_to_onnx(model:nn.Module, out_path):
    model.eval()
    
    # Needed for ONNX compatibility and doesn't affect output
    remove_sn(model)
    
    torch.onnx.export(model, 
                      (torch.rand(1, G_NCH, G_IMG_SZ, G_IMG_SZ),), 
                      out_path,
                      opset_version=11)

In [ ]:
export_to_onnx(learn_f2a.model, 'f2a_model.onnx')
export_to_onnx(learn_a2f.model, 'a2f_model.onnx')

## Run ONNX model using ONNX Runtime

You need to install ONNX and ONNX Runtime. Note that ONNX Runtime is compatible with Python versions 3.5 to 3.7.

In [ ]:
!pip install onnx
!pip install onnxruntime

In [ ]:
import onnx, onnxruntime
onnx_model = onnx.load("f2a_model.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
class ONNXInferenceEngine():
    def __init__(self, ort_session):
        self.ort_session = ort_session
        
    def _decode_onnx_out(self, onnx_preds):
        return ((onnx_preds[0] * G_NORM_STD + G_NORM_MEAN) * 255).astype(int)#int()
        
    def __call__(self, hwc_bgr_img):
        downsample = hwc_bgr_img.shape[0] > G_IMG_SZ
        # It's not clear what method is better when one dimension is bigger and the other is smaller,
        # so I'm only looking at height
        inter_method = cv2.INTER_AREA if downsample else cv2.INTER_LINEAR
        hwc_bgr_img = cv2.resize(hwc_bgr_img,
                                 dsize=(G_IMG_SZ, G_IMG_SZ),
                                 interpolation=inter_method)
        hwc_rgb_img = bgr_to_rgb(hwc_bgr_img)
        inp = quick_encode(hwc_rgb_img).numpy()
    
        ort_inputs = {self.ort_session.get_inputs()[0].name: inp}
        ort_outs = self.ort_session.run(None, ort_inputs) # -> 1 item list

        dec_ort_outs = self._decode_onnx_out(ort_outs[0]) 
        return dec_ort_outs
    
    @classmethod
    def from_model_path(cls, model_path):
        ort_session = onnxruntime.InferenceSession(model_path)
        return cls(ort_session)

Test ONNX Runtime with a sample image:

In [ ]:
onnx_rt_out = ONNXInferenceEngine.from_model_path("f2a_model.onnx")(sample_img)
TensorImage(onnx_rt_out).show()

Compare ONNX model output with Pytorch model output:

In [ ]:
np.abs(onnx_rt_out - torch_out).sum()

In [ ]:
np.where(np.abs(onnx_rt_out - torch_out) > 0)

# OpenVINO optimized models

[OpenVINO](https://docs.openvino.ai/latest/index.html) is an open-source toolkit for optimizing and deploying AI inference. 

Its inference engine **only supports Intel devices**. Moreover, it only accepts models with ONNX and OpenVINO IR formats.

The OpenVINO IR is an intermediate representation of a model adjusted for optimal performance when inferred with the Inference Engine. 

You can convert a model to IR using the [OpenVINO Model Optimizer](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html). For PyTorch models, you first need to convert the model to ONNX and then to OpenVINO IR. You could skip the last step but it's not recommended.

The IR of a model consists of a pair of files describing the model:
- .xml - Describes the network topology
- .bin - Contains the weights and biases binary data.

In [ ]:
def install_open_vino():
    !add-apt-repository -y ppa:deadsnakes/ppa
    !apt-get install -y python3.7-dev
    !python3 -m venv openvino_env
    !source openvino_env/bin/activate
    !python -m pip install --upgrade pip
    !pip install openvino-dev[onnx]

In [ ]:
install_open_vino()

In [ ]:
from openvino.inference_engine import IECore


class OpenVINOInferenceEngine():
    def __init__(self, model_path, weights_path=''):
        self.ie = IECore()
        net = self.ie.read_network(model=model_path, weights=weights_path)
        self.exec_net = self.ie.load_network(network=net, device_name="CPU")
        self.input_layer = next(iter(self.exec_net.input_info))
        self.output_layer = next(iter(self.exec_net.outputs))
        
    def _decode_out(self, onnx_preds):
        return ((onnx_preds[0] * G_NORM_STD + G_NORM_MEAN) * 255).astype(int)#int()
        
    def __call__(self, hwc_bgr_img):
        downsample = hwc_bgr_img.shape[0] > G_IMG_SZ
        # TODO: don't know what method is better when one dimension is bigger and the other is smaller,
        # so I'm only looking at height
        inter_method = cv2.INTER_AREA if downsample else cv2.INTER_LINEAR
        hwc_bgr_img = cv2.resize(hwc_bgr_img,
                                 dsize=(G_IMG_SZ, G_IMG_SZ),
                                 interpolation=inter_method)
        hwc_rgb_img = bgr_to_rgb(hwc_bgr_img)
        inp = quick_encode(hwc_rgb_img).numpy()
    
        preds = self.exec_net.infer(inputs={self.input_layer: inp})
        preds = preds[self.output_layer]

        dec_preds = self._decode_out(preds) 
        return dec_preds

## Inference with OpenVINO IE and ONNX model

In [ ]:
ov_onnx_out = OpenVINOInferenceEngine('f2a_model.onnx')(sample_img)
TensorImage(ov_onnx_out).show()

Compare with the output of the Pytorch model:

In [ ]:
np.abs(ov_onnx_out - torch_out).sum()

## Inference with OpenVINO IE and OpenVINO IR model

### Convert ONNX model to OpenVINO IR

In [ ]:
openvino_ir_model_path = Path('./openvino_model')
!mo --input_model f2a_model.onnx --output_dir $openvino_ir_model_path --batch 1

In [ ]:
!ls $openvino_ir_model_path

### Perform inference

In [ ]:
ov_out = OpenVINOInferenceEngine(openvino_ir_model_path/'f2a_model.xml')(sample_img)
TensorImage(ov_out).show()

Compare with PyTorch generated image:

In [ ]:
np.abs(ov_out - torch_out).sum()

# Test optimized model with a pipeline of face detection + conversion

## Face detection

In [ ]:
DETECTOR_TARGET_SZ = (300, 300)

In [ ]:
class CaffeFaceDetectorDnn:
    def __init__(self):
        # model structure
        !wget https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
        #pre-trained weights: 
        !wget https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel
        self.detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
        
    def __call__(self, inp):
        self.detector.setInput(inp)
        return self.detector.forward()
    
    
class OpenVINOFaceDetectorDnn():
    def __init__(self, model_path, weights_path=''):
        self.ie = IECore()
        net = self.ie.read_network(model=model_path, weights=weights_path)
        self.exec_net = self.ie.load_network(network=net, device_name="CPU")
        self.input_layer = next(iter(self.exec_net.input_info))
        self.output_layer = next(iter(self.exec_net.outputs))
        
    def __call__(self, inp):
        return self.exec_net.infer(inputs={self.input_layer: inp})[self.output_layer]


class OpenCVFaceDetector:
    def __init__(self, detector=None):
        self.detector = detector
    
    def _prepare_for_detector(self, img):
        resized_img = cv2.resize(img, DETECTOR_TARGET_SZ)
        tfm_aspect_ratio = [img.shape[i]/resized_img.shape[i] for i in (0, 1)]
        img_blob = cv2.dnn.blobFromImage(image=resized_img)
        return img_blob, tfm_aspect_ratio
    
    def _get_detector(self):
        if self.detector is None:
            self.detector = CaffeFaceDetectorDnn()
        return self.detector
    
    def detect(self, img:np.array, thresh=0.9):
        orig_img = img
        img, tfm_aspect_ratio = self._prepare_for_detector(img)
        detector = self._get_detector()
        detector_out = detector(img)
        detections = OpenCVDetectionList(detector_out, tfm_aspect_ratio, thresh=thresh)
        return detections


class OpenCVDetectionList():
    def __init__(self, detector_out, tfm_aspect_ratio, thresh=0.9):
        column_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]
        df = pd.DataFrame(detector_out[0][0], columns=column_labels)
        df = df[df.confidence > thresh]
        self.df = self._scale_coords_to_orig_sz(df, tfm_aspect_ratio)
        
    def _scale_coords_to_orig_sz(self, df, aspect_ratio):
        orig_width = int(DETECTOR_TARGET_SZ[1] * aspect_ratio[1])
        orig_height = int(DETECTOR_TARGET_SZ[0] * aspect_ratio[0])
        for _, det in df.iterrows():
            det.left = min((det.left * orig_width).astype(int), orig_width)
            det.top = min((det.top * orig_height).astype(int), orig_height)
            det.right = min((det.right * orig_width).astype(int), orig_width)
            det.bottom = min((det.bottom * orig_height).astype(int), orig_height)
        return df
        

def show_detected_images(source_img, det_list):
    for _, det in det_list.df.iterrows():
        l, t, r, b = [int(coord) for coord in (det.left, det.top, det.right, det.bottom)]
        is_empty = (l >= r) or (t >= b)
        if not is_empty: 
            show_cv_img(source_img[t:b, l:r])
        plt.figure()
        
        
def show_detected_boxes(source_img, det_list):
    img = source_img.copy()
    for _, det in det_list.df.iterrows():
        l, t, r, b = [int(coord) for coord in (det.left, det.top, det.right, det.bottom)]
        is_empty = (l >= r) or (t >= b)
        if not is_empty: 
            cv2.rectangle(img, (l, t), (r, b), (50, 50, 50), 1)
    show_cv_img(img)

Test the face detector:

In [ ]:
detector = OpenCVFaceDetector()
# detector2 = OpenCVFaceDetector(OpenVINOFaceDetectorDnn(
#     openvino_ir_model_path/'res10_300x300_ssd_iter_140000.xml',
#     openvino_ir_model_path/'res10_300x300_ssd_iter_140000.bin'
# ))

In [ ]:
img = get_img_from_path(ds_path.ls()[6])
show_cv_img(img)

In [ ]:
det_list = detector.detect(img, 0.9)
# det_list2 = detector2.detect(img, 0.9)
det_list.df #, det_list2.df

If you have any issues with pandas, run: `!pip install pandas==1.3.5`

In [ ]:
show_detected_images(img, det_list)

In [ ]:
show_detected_boxes(img, det_list)

## (Optional) Convert Caffe face detection model to OpenVINO IR

In [ ]:
!mo --input_model res10_300x300_ssd_iter_140000.caffemodel --input_proto deploy.prototxt --output_dir $openvino_ir_model_path

## Ensemble detect + replace

In [ ]:
def expand_box(ltrb_coords, orig_shape, extra_w_pct=0.25, extra_h_pct=0.25,
               enforce_symmetry=False):
    # bottom and right coordinates must be interpreted as the first indexes out
    # of the image, so that numpy indexing logic is matched and one would index 
    # img[t:b, l:r]
    l, t, r, b = ltrb_coords
    orig_h, orig_w, *_ = orig_shape
    w = r - l
    h = b - t
    
    extra_w = w * extra_w_pct
    horizontal_inc = (min(round(extra_w/2), l, orig_w - r) if enforce_symmetry else round(extra_w/2))
    l = max(0, l - horizontal_inc)
    r = min(orig_w, r + horizontal_inc)
    
    extra_h = h * extra_h_pct
    vertical_inc = (min(round(extra_h/2), t, orig_h - b) if enforce_symmetry else round(extra_h/2))
    t = max(0, t - vertical_inc)
    b = min(orig_h, b + vertical_inc)
       
    return l, t, r, b
  

assert expand_box((5, 6, 65, 66), (100, 100)) == (0, 0, 73, 74)
assert expand_box((5, 6, 65, 66), (100, 100), enforce_symmetry=True) == (0, 0, 70, 72)
assert expand_box((35, 36, 95, 96), (100, 100)) == (27, 28, 100, 100)
assert expand_box((35, 36, 95, 96), (100, 100), enforce_symmetry=True) == (30, 32, 100, 100)


def pad_to_make_square(img:np.array, pad_mode='edge', **pad_kwargs):
    h, w, *_ = img.shape
    if h == w: return img, (0, 0, w, h)
    
    target_sz = max(h, w)
    if target_sz == h:
        vertical_pad = (0, 0)
        horizontal_inc = target_sz - w
        horizontal_pad = (horizontal_inc//2, horizontal_inc//2 + horizontal_inc%2)
    else:
        horizontal_pad = (0, 0)
        vertical_inc = target_sz - h
        vertical_pad = (vertical_inc//2, vertical_inc//2 + vertical_inc%2)
    extra_axis = len(img.shape) - 2
    padded_img = np.pad(img, (vertical_pad, horizontal_pad, *[(0, 0)]*extra_axis), 
                        mode=pad_mode, **pad_kwargs)
    orig_img_ltrb_coords = (horizontal_pad[0], 
                            vertical_pad[0], 
                            horizontal_pad[0] + w,
                            vertical_pad[0] + h)
    return padded_img, orig_img_ltrb_coords


def test_pad_to_make_square():
    in_row_v = np.array([[1, 2, 3]])
    expected_padded_row_v = np.array([[0]*3, [1, 2, 3], [0]*3])
    expected_row_v_coords = (0, 1, 3, 2)
    actual_padded_row_v, actual_row_v_coords = pad_to_make_square(in_row_v, pad_mode='constant', constant_values=0)
    assert (actual_padded_row_v == expected_padded_row_v).all()
    assert actual_row_v_coords == expected_row_v_coords
    
    in_col_v = np.array([[1], [2], [3]])
    expected_padded_col_v = np.array([[0, 1, 0], [0, 2, 0], [0, 3, 0]])
    expected_col_v_coords = (1, 0, 2, 3)
    actual_padded_col_v, actual_col_v_coords = pad_to_make_square(in_col_v, pad_mode='constant', constant_values=0)
    assert (actual_padded_col_v == expected_padded_col_v).all()
    assert actual_col_v_coords == expected_col_v_coords
    

test_pad_to_make_square()
        

def transform_img(inference_engine, img, detected_faces):
    out_img = img.copy()
    # Iter in reversed order to ensure the most confident appears on top in case of overlapping
    for _, det in detected_faces.df[::-1].iterrows():
        l, t, r, b = [int(coord) for coord in (det.left, det.top, det.right, det.bottom)]        
        is_empty = (l >= r) or (t >= b)
        if not is_empty: 
            l, t, r, b = expand_box((l, t, r, b), img.shape, extra_w_pct=0.4, extra_h_pct=0.4)
            face_subimg = img[t:b, l:r]
            
            # Make square to prevent fastai from cropping interesting parts of the image
            face_subimg, coords_before_pad = pad_to_make_square(face_subimg)
            
            pred = inference_engine(face_subimg)
            
            #pred = pred[[2, 1, 0]].permute(1, 2, 0).numpy().astype(np.uint8)
            pred_hwc_bgr = pred[[2, 1, 0]].transpose(1, 2, 0).astype(np.uint8)
            pred_hwc_bgr = cv2.resize(pred_hwc_bgr,
                                      dsize=(face_subimg.shape[1], face_subimg.shape[0]),
                                      interpolation=cv2.INTER_LINEAR)
            pred_hwc_bgr = pred_hwc_bgr[coords_before_pad[1]:coords_before_pad[3], coords_before_pad[0]:coords_before_pad[2]]
            #plt.figure()
            #show_cv_img(pred_np)
            out_img[t:b, l:r, :] = pred_hwc_bgr
            
    return out_img


def convert_img(inference_engine, img, detector=None, det_thresh=0.9):      
    if detector is None: detector = OpenCVFaceDetector()
    det_list = detector.detect(img, 0.9) 
    return transform_img(inference_engine, img, det_list)


def convert_video(learn, vid_path, out_path, detector=None, det_thresh=0.9):
    if detector is None: detector = OpenCVFaceDetector()
        
    vid_capture = cv2.VideoCapture(vid_path)
    output = None

    if (vid_capture.isOpened() == False):
        print("Error opening the video file")
    else:
        fps = vid_capture.get(cv2.CAP_PROP_FPS)
        print('Frames per second : ', fps,'FPS')

        frame_count = vid_capture.get(cv2.CAP_PROP_FRAME_COUNT)
        print('Frame count : ', frame_count)
        
        frame_width = int(vid_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(vid_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_size = (frame_width,frame_height)

        output = cv2.VideoWriter(out_path, 
                                 cv2.VideoWriter_fourcc('M','J','P','G'), 
                                 20, 
                                 frame_size)

    frame_idx = 0
    while(vid_capture.isOpened()):
        ok, frame = vid_capture.read()
        if ok:
            #cv2.imshow('Frame',frame)
            #show_cv_img(frame)
            #plt.figure()
            
            converted_frame = convert_img(learn, frame, detector=detector, det_thresh=det_thresh)
            
            output.write(converted_frame)
            if (frame_idx + 1) % 50 == 0: print('Written frame ', frame_idx)
        else:
            print('Error in frame ', frame_idx)
            break
        frame_idx += 1

    vid_capture.release()
    if output is not None: output.release()

In [ ]:
detector = OpenCVFaceDetector()

Test pipeline with an example image:

In [ ]:
test_out_img = convert_img(#ONNXInferenceEngine.from_model_path('f2a_model.onnx'), 
                           #TorchQuickInferenceEngine.from_fastai_learn(learn_f2a),
                           FastaiInferenceEngine(learn_f2a),
                           get_img_from_path(celeba_path.ls()[0]),
                           #get_img_from_url('https://[REPLACE WITH URL OF EXAMPLE IMAGE]'), 
                           detector=detector)
show_cv_img(test_out_img)

Test pipeline with an example video:

In [ ]:
convert_video(learn_f2a, '[REPLACE WITH PATH OF A SAMPLE VIDEO]', './test_out1.avi', 
              detector=detector,
              det_thresh=0.5)

## Performance comparison

In [ ]:
test_img = get_img_from_path(celeba_path.ls()[0])
show_cv_img(test_img)
test_img.shape

In [ ]:
fastai_inf_eng = FastaiInferenceEngine(learn_f2a)
%timeit -r 5 -n 3 with learn_f2a.no_bar(): convert_img(fastai_inf_eng, test_img, detector=detector)

In [ ]:
torch_inf_eng = TorchQuickInferenceEngine.from_fastai_learn(learn_f2a)
%timeit -r 5 -n 3 convert_img(torch_inf_eng, test_img, detector=detector)

In [ ]:
onnx_inf_eng = ONNXInferenceEngine.from_model_path('f2a_model.onnx')
%timeit -r 5 -n 3 convert_img(onnx_inf_eng, test_img, detector=detector)

In [ ]:
ov_onnx_inf_eng = OpenVINOInferenceEngine('f2a_model.onnx')
%timeit -r 5 -n 3 convert_img(ov_onnx_inf_eng, test_img, detector=detector)

In [ ]:
ov_inf_eng = OpenVINOInferenceEngine(openvino_ir_model_path/'f2a_model.xml')
%timeit -r 5 -n 3 convert_img(ov_inf_eng, test_img, detector=detector)

To get a detailed report of the execution times by method, use `%prun`. For instance:

In [ ]:
%prun -s cumulative convert_img(fastai_inf_eng, test_img, detector=detector)